In [1461]:
# Imports
import numpy as np
import random

In [2495]:
# Set-up

# Parameters
genome_size = 16
max_group_size = 6
starting_group_size = 4
group_number = 4
mutation_rate = 0.5
group_split_rate = 0

# Create initial population
initial_population = np.random.randint(0, 2, size=(group_number, max_group_size, genome_size), dtype=np.int8)
population = initial_population.copy()
display(population)

# Keep track of group sizes
group_sizes = np.full((group_number), starting_group_size, dtype = np.int8)
display(group_sizes)

array([[[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0],
        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1]],

       [[1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1],
        [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1],
        [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
        [0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0]],

       [[1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
        [0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
        [0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
        [1

array([4, 4, 4, 4], dtype=int8)

In [2496]:
# Fitness table

# Take population array and calculate the fitness for each indiviual in the initial population
fitness_table = np.zeros((group_number, max_group_size, 1))

# 1. Calculate the mean along the genome axis (axis 2)
# Keepdims ensures the shape remains (group_number, max_group_size, 1)
fitness_table = np.mean(population, axis=2, keepdims=True)

# 2. (Optional) Zero out fitness for "empty" slots in the groups
# This ensures that individuals beyond the group_size don't contribute to group stats
for i, size in enumerate(group_sizes):
    fitness_table[i, size:] = 0

fitness_array = fitness_table.flatten()
display(fitness_array)


# If a group splits or a individual is born, recalculate the fitnesses in that/those group(s)
# Than update this table 
# To not calculate the fitnesses of each individuals each timestep but only SOME individuals SOMEtimes

array([0.25  , 0.625 , 0.4375, 0.375 , 0.    , 0.    , 0.4375, 0.5625,
       0.6875, 0.5625, 0.    , 0.    , 0.5   , 0.4375, 0.25  , 0.5625,
       0.    , 0.    , 0.5   , 0.5   , 0.625 , 0.5625, 0.    , 0.    ])

In [2497]:
# Function for when a group is full
def groupFullEvent(pop_input, group_id_input):
    
    # If one dies 
    if random.random() > group_split_rate:
        last_row_index = max_group_size - 1
        # Select unlucky member
        unlucky_index = np.random.choice(max_group_size)
        # Swap unlucky with last row
        pop_input[group_id_input, [unlucky_index, last_row_index]] = pop_input[group_id_input, [last_row_index, unlucky_index]]
        # Move fitness value with swapped individual
        moved_fitness = fitness_array[(group_id_input*max_group_size)+last_row_index]
        fitness_array[(group_id_input*max_group_size)+unlucky_index] = moved_fitness
        
        # Decrease group size to 'remove' member
        group_sizes[group_id_input] += -1
        print(f"Individual {unlucky_index} in group {group_id_input} died")
        return pop_input
    
    # If group splits:
    # Determine group that gets removed
    else:
        # All groups except the current one
        left_over_groups = [i for i in range(group_number) if i != group_id_input]
        unlucky_group_index = np.random.choice(left_over_groups)
        # Set group size of unlucky group to 0
        group_sizes[unlucky_group_index] = 0

        # Boolean mask to split the group in a random way
        move_mask = np.random.choice([True, False], size = max_group_size, p = [0.5, 0.5])
        
        if not np.any(move_mask): # If all false, select 1 random to be true
            move_mask[np.random.randint(0, max_group_size)] = True
        elif np.all(move_mask):   # Vice versa
            move_mask[np.random.randint(0, max_group_size)] = False
            
        # Move those marked 'True' to new group
        moving_members = pop_input[group_id_input, move_mask, :]
        num_moving = len(moving_members)
        # Place them at the top of the new group
        pop_input[unlucky_group_index, 0:num_moving, :] = moving_members
        # Set group size to new amount
        group_sizes[unlucky_group_index] = num_moving
        
        
        # Move those marked 'False' to top of old group
        staying_members = pop_input[group_id_input, ~move_mask, :]
        num_staying = len(staying_members)
        # Place on top
        pop_input[group_id_input, 0:num_staying, :] = staying_members
        # Set group size to new amount
        group_sizes[group_id_input] = num_staying

        # ALSO RECALCULATE FITNESS
        
        # Debug
        print(f"Group {group_id_input} split and replaced group {unlucky_group_index}.")
        print(f"{num_moving} individuals moved. {num_staying} stayed.")
        
        return pop_input

In [2498]:
# Function for reproduction

def reproductionEvent(pop_input):
    # Select a random individual from fitness array based on weight
    probabilities = fitness_array / np.sum(fitness_array)
    selected_index = np.random.choice(len(fitness_array), p=probabilities)
    # Get group and member id
    group_id = selected_index//max_group_size
    member_id = selected_index%max_group_size

    print(f'Selected member {member_id} of group {group_id}')

    # Get genome of selected individual
    selected_genome = pop_input[group_id, member_id]
    # Get fitness of selected memeber
    copied_fitness = fitness_array[(group_id*max_group_size)+member_id]

    print(f'selected genome = {selected_genome}')

    # Check if group is full
    if group_sizes[group_id] == max_group_size:
        print('Group is full, performing groupFullEvent')
        # Activate groupFullEvent
        groupFullEvent(pop_input, group_id)
    else:
        print('Group still has space')

    # Copy genome for reproduction
    genome_copy = selected_genome.copy()

    # Mutation with certain mutation rate (added later)
    # if random.random() < mutation_rate: 
    #   Flip one random locus
    
    # Place gemone copy below the taken spaces, overiting the 'garbage' data
    pop_input[group_id, group_sizes[group_id]] = genome_copy
    print(f"Reproduced member in group {group_id} at slot {group_sizes[group_id]}")

    # If mutated, recalculate fitness. Otherwise just copy fitness
    # Take fitness of copied member
    print('fitness_array before:')
    display(fitness_array)
    fitness_array[((group_id*max_group_size)+group_sizes[group_id])] = copied_fitness
    print('Fitness array after:')
    display(fitness_array)
    
    # Increase group size so new individual is considered as 'actual' data
    group_sizes[group_id] += 1
    display(group_sizes)

    return pop_input

In [2777]:
reproductionEvent(population)

Selected member 1 of group 2
selected genome = [1 0 1 0 0 0 1 1 0 1 0 0 1 1 1 0]
Group is full, performing groupFullEvent
Individual 5 in group 2 died
Reproduced member in group 2 at slot 5
fitness_array before:


array([0.4375, 0.4375, 0.4375, 0.4375, 0.4375, 0.4375, 0.5625, 0.5625,
       0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625,
       0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625])

Fitness array after:


array([0.4375, 0.4375, 0.4375, 0.4375, 0.4375, 0.4375, 0.5625, 0.5625,
       0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625,
       0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625, 0.5625])

array([6, 6, 6, 6], dtype=int8)

array([[[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]],

       [[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
        [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
        [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
        [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
        [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
        [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0]],

       [[1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
        [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
        [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
        [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
        [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
        [1